In [1]:
## UI
# general imports
import os
import logging
import numpy as np
import time
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import constants


# models
from src.models import t2i_recsys
from src.models.avg_embeddings_model import AvgEmbeddings
from src.models.soft_cosine_model import SoftCosine
from src.models.knn_model import KNN
# from src.models.USE_model import USE_Recsys
from src.nlp_util.textacy_util import extract_textrank_from_text

# API and UI files
from src import api_helper
from src import tagging_api

In [2]:
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import constants

In [208]:
import pandas as pd
import numpy as np

#get training path
train_dir = constants.TRAIN_DIR
clean_dir = constants.CLEAN_DIR
art_prefix = constants.Text_Prefix
img_prefix = constants.Media_Prefix
#tag types
tag_types = ['org', 'place', 'subject','person']
# article_summary = pd.read_csv(f'{train_dir}/{art_prefix}summary.csv')
article_summary = pd.read_csv(f'{constants.CLEAN_DIR}/article_summary.csv')
image_summary =  pd.read_csv(f'{constants.CLEAN_DIR}/{img_prefix}summary.csv')
#preview dir
preview_dir = f'{constants.DATA_DIR}/preview'

In [6]:
sample = pd.read_csv("/Users/pwong/Downloads/data_sample2.csv")
sample.shape

(500, 8)

In [189]:
def article_images(id, data):
    """
    Gets all the images for a particular article id
    """
#     csv_file = constants.CLEAN_DIR / 'image_summary.csv'
#     data = pd.read_csv(csv_file)
    subset = data[['id', 'article_idx']]
    images = subset[subset['article_idx'] == id]['id'].values
    return images

In [210]:
true_img = []
for id in sample['ids'].values:
    true_img.append(article_images(id, image_summary))

In [ ]:
y_true = sample['true_imgs'][0]
y_pred_knn = sample['knn'][0]

In [24]:
def clean_up_true(y):
    y = y.replace('[', '').replace(']', '').replace('"', '').strip().split()
    y = [b.replace('"', '').strip()[1:-1] for b in y]
    return y

In [25]:
def clean_up_pred(y):
    y = y.replace('[', '').replace(']', '').replace('"', '').strip().split(',')
    y = [b.replace('"', '').strip()[1:-1] for b in y]
    return y

In [ ]:
y_pred = clean_up_pred(y_pred)
y_true = clean_up_true(y_true)

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
y_true = ['a','b']
y_pred = ['a', 'c', 'd']

In [26]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def recall(y_true, y_pred):
    num_true_pos = len(intersection(y_true, y_pred)) # number of true positive
    num_true_image = len(y_true) # number of true images used in the article
    score = num_true_pos/num_true_image
    return score

def precision(y_true, y_pred):
    num_true_pos = len(intersection(y_true, y_pred)) # number of true positive
    num_pred = len(y_pred) # number of predicted images
    score = num_true_pos/num_pred 
    return score

In [27]:
def r_precision(y_true, y_pred):
    num_true_image = len(y_true) # number of true images used in the article
    num_true_pos = len(intersection(y_true, y_pred[:num_true_image])) # number of true positive
    
    score = num_true_pos/num_true_image
    return score    

In [94]:
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(y_true, y_pred, k=len(y_true), method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    r = [1* (true_img in y_pred[:len(y_true)]) for true_img in y_true] # 1 if in true images
#     print("r", r)
    dcg_max = dcg_at_k(np.ones(len(y_true)), k = k) # IDCG with ground truth 
#     print('IDCG:', dcg_max)
#     dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

## Model Comparison

In [218]:
list(map(len, true_img))

[7,
 1,
 1,
 1,
 1,
 14,
 10,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 2,
 3,
 2,
 6,
 10,
 1,
 2,
 1,
 10,
 9,
 1,
 2,
 4,
 7,
 1,
 1,
 2,
 2,
 1,
 4,
 1,
 1,
 4,
 5,
 2,
 10,
 2,
 1,
 1,
 3,
 5,
 1,
 6,
 1,
 18,
 3,
 7,
 3,
 1,
 1,
 9,
 2,
 2,
 10,
 5,
 2,
 1,
 5,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 1,
 1,
 3,
 5,
 8,
 3,
 1,
 1,
 11,
 4,
 1,
 6,
 8,
 6,
 4,
 2,
 7,
 3,
 3,
 2,
 1,
 9,
 2,
 5,
 1,
 1,
 7,
 2,
 8,
 2,
 10,
 8,
 5,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 9,
 4,
 6,
 1,
 1,
 1,
 1,
 1,
 2,
 12,
 2,
 12,
 3,
 2,
 2,
 2,
 13,
 6,
 1,
 4,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 3,
 2,
 12,
 1,
 9,
 1,
 7,
 5,
 1,
 2,
 1,
 1,
 4,
 4,
 2,
 1,
 1,
 2,
 12,
 2,
 5,
 4,
 3,
 1,
 5,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 9,
 6,
 3,
 3,
 1,
 13,
 8,
 12,
 5,
 1,
 6,
 1,
 2,
 1,
 2,
 2,
 1,
 3,
 2,
 2,
 7,
 5,
 3,
 1,
 8,
 1,
 7,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 2,
 5,
 1,
 2,
 1,
 3,
 5,
 1,
 2,
 5,
 2,
 1,
 5,
 2,
 5,
 1,
 1,
 3,
 1,
 2,
 3,
 1,
 2,
 1,
 2,
 3,
 2,
 4,
 1,
 1,

In [211]:
recall_score = []
models = ['knn', 't2t', 'softcos', 'emb', 'use']
for model in models:
    for ix in range(sample.shape[0]):
#         y_true = sample['true_imgs'][ix]
        y_true = list(true_img[ix])
        y_pred = sample[model][ix]

        y_pred = clean_up_pred(y_pred)
#         y_true = clean_up_true(y_true)
        recall_score.append(recall(y_true, y_pred))

In [212]:
recall_score_all = np.mean(recall_score[:500]), np.mean(np.array(recall_score)[500:1000]), np.mean(np.array(recall_score)[1000:1500]), np.mean(np.array(recall_score)[1500:2000]), np.mean(np.array(recall_score)[2000:2500])

recall_score_all

(0.430395772005772,
 0.0,
 0.2996012449538765,
 0.3591600919548288,
 0.511206318067897)

In [224]:
precision_score = []
models = ['knn', 't2t', 'softcos', 'emb', 'use']
for model in models:
    for ix in range(sample.shape[0]):
        y_true = list(true_img[ix])
        y_pred = sample[model][ix]

        y_pred = clean_up_pred(y_pred)
#         y_true = clean_up_true(y_true)
        precision_score.append(precision(y_true, y_pred))
        
precision_score_all = np.mean(precision_score[:500]), np.mean(np.array(precision_score)[500:1000]), np.mean(np.array(precision_score)[1000:1500]), np.mean(np.array(precision_score)[1500:2000]),  np.mean(np.array(precision_score)[2000:2500])
precision_score_all

(0.023, 0.0, 0.016966666666666668, 0.01993333333333333, 0.026466666666666666)

In [225]:
r_precision_score = []
models = ['knn', 't2t', 'softcos', 'emb', 'use']
for model in models:
    for ix in range(sample.shape[0]):
        y_true =list(true_img[ix])
        y_pred = sample[model][ix]

        y_pred = clean_up_pred(y_pred)
#         y_true = clean_up_true(y_true)
        r_precision_score.append(r_precision(y_true, y_pred))
        
r_precision_score_all = np.mean(r_precision_score[:500]), np.mean(np.array(r_precision_score)[500:1000]), np.mean(np.array(r_precision_score)[1000:1500]), np.mean(np.array(r_precision_score)[1500:2000]), np.mean(np.array(r_precision_score)[2000:2500])
r_precision_score_all

(0.16751856365856366,
 0.0,
 0.10983512265512266,
 0.11039796320638426,
 0.24324632542895702)

In [226]:
ndcg_k_score = []
models = ['knn', 't2t', 'softcos', 'emb', 'use']
for model in models:
    for ix in range(sample.shape[0]):
        y_true = list(true_img[ix])
        y_pred = sample[model][ix]

        y_pred = clean_up_pred(y_pred)
#         y_true = clean_up_true(y_true)
        ndcg_k_score.append(ndcg_at_k(y_true, y_pred))
        
ndcg_k_score_all = np.mean(ndcg_k_score[:500]), np.mean(np.array(ndcg_k_score)[500:1000]), np.mean(np.array(ndcg_k_score)[1000:1500]), np.mean(np.array(ndcg_k_score)[1500:2000]), np.mean(np.array(ndcg_k_score)[2000:2500])
ndcg_k_score_all

(0.16793572699250592,
 0.0,
 0.11007184992054492,
 0.11157565501098272,
 0.24533416865447322)

In [227]:
score_all_df = pd.DataFrame({'models':['knn', 't2t', 'softcos', 'emb', 'use'], 
                            'recall': recall_score_all,
                           'precision': precision_score_all,
                           'r_precision': r_precision_score_all,
                           'ndcg_at_k': ndcg_k_score_all})
score_all_df

,models,recall,precision,r_precision,ndcg_at_k
0,knn,0.430396,0.023000,0.167519,0.167936
1,t2t,0.000000,0.000000,0.000000,0.000000
2,softcos,0.299601,0.016967,0.109835,0.110072
3,emb,0.359160,0.019933,0.110398,0.111576
4,use,0.511206,0.026467,0.243246,0.245334


In [228]:
np.round(score_all_df, 2)

,models,recall,precision,r_precision,ndcg_at_k
0,knn,0.43,0.02,0.17,0.17
1,t2t,0.00,0.00,0.00,0.00
2,softcos,0.30,0.02,0.11,0.11
3,emb,0.36,0.02,0.11,0.11
4,use,0.51,0.03,0.24,0.25


In [222]:
score_df = pd.DataFrame({'article_id': sample['ids'].values,'num_true_imgs': list(map(len, true_img)), 
                         'knn':np.array(recall_score)[0:500], 
              't2t':np.array(recall_score)[500:1000],
             'softcos': np.array(recall_score)[1000:1500], 
             'emb': np.array(recall_score)[1500:2000],
             'use': np.array(recall_score)[2000:2500]})
score_df

,article_id,num_true_imgs,knn,t2t,softcos,emb,use
0,6996d6ecdeb546949a8ab082dd4c937b,7,0.142857,0.0,0.000000,0.285714,0.000000
1,4347de95b5e543df8accdd67eea7c910,1,1.000000,0.0,0.000000,1.000000,1.000000
2,7b5cf4eec7d547508b7de4f34d3b7f35,1,0.000000,0.0,0.000000,1.000000,0.000000
3,020b2c22715640d99dc4573cb196ecc0,1,0.000000,0.0,0.000000,1.000000,1.000000
4,c3310c4dcc454ffdbb5390e17148ad86,1,0.000000,0.0,0.000000,0.000000,1.000000
5,29f6bffe7a93438dbafb3461ecb75b30,14,0.071429,0.0,0.071429,0.000000,0.000000
6,72a66c8aefc047848eaaec289e740b32,10,0.100000,0.0,0.100000,0.600000,0.900000
7,dc16b243eb024116aacabe5eb34a40bd,1,0.000000,0.0,0.000000,0.000000,1.000000
8,f01c7a933df947d9aa7156064eb2e166,2,1.000000,0.0,1.000000,0.000000,0.000000
9,2feb23df63b24697bc9a405d07825536,1,0.000000,0.0,0.000000,0.000000,0.000000


In [ ]:
sample['id'] 

## NOT USE##

In [16]:
sample.columns

Index(['Unnamed: 0', 'ids', 'true_imgs', 'knn', 't2t', 'softcos', 'emb',
       'use'],
      dtype='object')

In [57]:
IDs = ['knn', 't2t', 'softcos', 'emb', 'use']
Defaults = {'recall': '', 'precision': '', 'r_precision': '', 'ndcg':''}

score_dict = dict.fromkeys(IDs, Defaults)

In [58]:
score_dict.keys()

dict_keys(['knn', 't2t', 'softcos', 'emb', 'use'])

In [20]:
score_dict['knn']['recall']

''

In [7]:
y_true = sample['true_imgs'][0]
y_pred_knn = sample['knn'][0]


In [54]:
sample.shape

(500, 8)

In [79]:
ix = 0
y_true = sample['true_imgs'][ix]
y_pred = sample[model][ix]

y_pred = clean_up_pred(y_pred)
y_true = clean_up_true(y_true)

In [81]:
len(y_pred)

60

In [95]:
metrics_func = [recall, precision, r_precision, ndcg_at_k]

In [96]:
for metric in metrics_func:
    print(metric)
    print(metric(y_true, y_pred))

<function recall at 0x1c74667598>
0.0
<function precision at 0x1c746677b8>
0.0
<function r_precision at 0x1c74667840>
0.0
<function ndcg_at_k at 0x1c6c85a048>
0.0


In [97]:
## Dictionary to keey score
IDs = ['knn', 't2t', 'softcos', 'emb', 'use']
Defaults = {'recall': '', 'precision': '', 'r_precision': '', 'ndcg_at_k':''}
score_dict = dict.fromkeys(IDs, Defaults)

model = 'knn'
# metrics = ['recall', 'precision', 'r_precision', 'ndcg_at_k']
metrics_func = [recall, precision, r_precision, ndcg_at_k]

for ix in range(sample.shape[0]):
    y_true = sample['true_imgs'][ix]
    y_pred = sample[model][ix]

    y_pred = clean_up_pred(y_pred)
    y_true = clean_up_true(y_true)
    
    # different metrics
    for ix, metric in enumerate(metrics_func):
        try:
            score_dict[model][ix].append(metric(y_true, y_pred))
        except:
            score_dict[model][ix] = []
            score_dict[model][ix].append(metric(y_true, y_pred))
        

model = 't2t'
for ix in range(sample.shape[0]):
    y_true = sample['true_imgs'][ix]
    y_pred = sample[model][ix]

    y_pred = clean_up_pred(y_pred)
    y_true = clean_up_true(y_true)
    # different metrics
    for ix, metric in enumerate(metrics_func):
        try:
            score_dict[model][ix].append(metric(y_true, y_pred))
        except:
            score_dict[model][ix] = []
            score_dict[model][ix].append(metric(y_true, y_pred))
            
model = 'softcos'
for ix in range(sample.shape[0]):
    y_true = sample['true_imgs'][ix]
    y_pred = sample[model][ix]

    y_pred = clean_up_pred(y_pred)
    y_true = clean_up_true(y_true)

    # different metrics
    for ix, metric in enumerate(metrics_func):
        try:
            score_dict[model][ix].append(metric(y_true, y_pred))
        except:
            score_dict[model][ix] = []
            score_dict[model][ix].append(metric(y_true, y_pred))
            
model = 'emb'
for ix in range(sample.shape[0]):
    y_true = sample['true_imgs'][ix]
    y_pred = sample[model][ix]

    y_pred = clean_up_pred(y_pred)
    y_true = clean_up_true(y_true)

    # different metrics
    for ix, metric in enumerate(metrics_func):
        try:
            score_dict[model][ix].append(metric(y_true, y_pred))
        except:
            score_dict[model][ix] = []
            score_dict[model][ix].append(metric(y_true, y_pred))
            
model = 'use'
for ix in range(sample.shape[0]):
    y_true = sample['true_imgs'][ix]
    y_pred = sample[model][ix]

    y_pred = clean_up_pred(y_pred)
    y_true = clean_up_true(y_true)

    # different metrics
    for ix, metric in enumerate(metrics_func):
        try:
            score_dict[model][ix].append(metric(y_true, y_pred))
        except:
            score_dict[model][ix] = []
            score_dict[model][ix].append(metric(y_true, y_pred))

In [102]:
score_dict['knn'].keys()

dict_keys(['recall', 'precision', 'r_precision', 'ndcg_at_k', 0, 1, 2, 3])

In [105]:
len(score_dict['knn'][0])

2500

In [104]:
np.mean(score_dict['knn'][0])

0.3200726853964748

In [70]:
np.mean(score_dict['knn']['recall'])

0.3200726853964748

In [62]:
for model in models:
    print(f'{model} average Recall')
    print(np.mean(score_dict[model]['recall']))
    
    print(f'{model} average Precision')
    print(np.mean(score_dict[model]['precision']))
    
    

knn average Recall
0.3200726853964748
knn average Precision
0.3200726853964748
t2t average Recall
0.3200726853964748
t2t average Precision
0.3200726853964748
softcos average Recall
0.3200726853964748
softcos average Precision
0.3200726853964748
emb average Recall
0.3200726853964748
emb average Precision
0.3200726853964748
use average Recall
0.3200726853964748
use average Precision
0.3200726853964748


In [10]:
sample.head()

,Unnamed: 0,ids,true_imgs,knn,t2t,softcos,emb,use
0,0,6996d6ecdeb546949a8ab082dd4c937b,['96b306693ffd46af94d50ea45063a8d3' 'ee86fc509...,"['a7852710bcf64360b96254ea87ea8f51', '472fd1de...","['582a6e095c674ffe95e5b8dc78bbb3fe', 'c1dd12f9...","['adf4dc2271de4d658c92ce138a08fb16', 'aba7f286...","['4a65af1b63ce45268ad52e909474d71a', '079a759d...","['da1a8451fb8d4d27a182c595c01026e5', 'd387596a..."
1,1,4347de95b5e543df8accdd67eea7c910,['039acf7392664113bf07f4d51899bd77'],"['2adcd45a195b41d79b10d2fef5e4e62b', 'b82af52d...","['f1c14f2d8e2c4158b7c8dd45027cde9a', 'e65d8251...","['af8faf417c5a4db3975c8a8e8765fd7c', 'fdf76dac...","['039acf7392664113bf07f4d51899bd77', '56a34eec...","['b948b1b0e13d4943982857d248f24aa1', '5c5ee573..."
2,2,7b5cf4eec7d547508b7de4f34d3b7f35,['dd3eddf54d1a4e87b184b988d176983b'],"['0d3582738c9f4440918d8c09f37d9706', '248293e3...","['b910f751e3984b3e8e875fbf528fd0c6', '30efe3f2...","['2a4165f73cb8473f87ed2f29e434b0aa', '3ac7677d...","['9255da4dea1846fcb451f5d125171890', '979c548b...","['f749145ef0e24d9987adb3857a3e5df9', 'f241a92e..."
3,3,020b2c22715640d99dc4573cb196ecc0,['a0971ea8e43b4f03828dbc745e3f6d68'],"['a59768460cea4e1398058ad8f136e2ff', '7059af1c...","['fdfb495e4eca4156a8d8d75ae410f8af', 'fbf3f089...","['85242f5193964027b8cf13a1141b9683', '628f2fc1...","['5586f0d53c6f40508af1c283a545d812', 'e2f9290f...","['a0971ea8e43b4f03828dbc745e3f6d68', 'e1953504..."
4,4,c3310c4dcc454ffdbb5390e17148ad86,['aa92bc6addd3452e95f306aa071b0d4b'],"['ea274e43ee1b4859b48a487a2b19393c', '1de02346...","['27ddb32b9213462fa24875acec7e083d', '72ba1337...","['244ba375f4cf4eb481e242a0c5aead3e', '4995468e...","['cafead6cdfd14b88aacc3517b9d13306', 'dc8baf52...","['aa92bc6addd3452e95f306aa071b0d4b', '2b541c5f..."


In [9]:
y_pred_knn

"['a7852710bcf64360b96254ea87ea8f51', '472fd1de7d1e46b0940327d16dfa9f4d', '46427e475c9c4f1ab47acdc881381f68', 'f427292978ee4226aa17245ec03a38d8', '91b9cb3a87e04222a75c40fcc6c95982', '310ddf79a01a4ec581b6964305aaaafd', '3917785d7e23420391134b687194ba2a', '484342c3a02549a1805832623f88804e', '6b32c35e860e4dab81a2829bc3058fc4', '68be0542042c47c1a0cddf641778da5b', '13ff75ffe00d4825a8c087a379e35580', '38bf76f90d4f482d929dc616f0f99ee7', '1be3205e1ec7459985525b918c69334b', 'c93ef2499f7e4e33b6b048af884fbdb1', 'f6e6bcab85ef4272a730ae92fd8e2bdf', '05fd4769141c4d13ab5879ed76c03acb', '2e9257707aa9438a8aae84a29ce4ea1d', '582a6e095c674ffe95e5b8dc78bbb3fe', '2c7d6dc435694006be269bd6f4e1f4e8', '2d3d6e3c33024a6c8ed7cd60a34a6f48', '52bd51e2e31b4358901961ff0737e707', 'f69a670b05eb47caa9aa5168a430630e', '8b710e0e10a747958e11b2055fca6f05', '792ba053b0884fea85833a68b3eac25e', '3cc1384e266d4adfa4014138b116513f', '8255ad8f3f4c42a886c6c508a97747b4', 'aa8b029e22194807949e57344e20734c', 'de9a9e612dda4414989917ac4

In [ ]:
y_pred = clean_up_pred(y_pred)
y_true = clean_up_true(y_true)